In [1]:
import pandas as pd
import numpy as np
import time

In [24]:
ratings = pd.read_csv("./ml-latest-small/ratings.csv")
ratings[ratings.userId==2].head(20)

,userId,movieId,rating,timestamp
232,2,318,3.0,1445714835
233,2,333,4.0,1445715029
234,2,1704,4.5,1445715228
235,2,3578,4.0,1445714885
236,2,6874,4.0,1445714952
237,2,8798,3.5,1445714960
238,2,46970,4.0,1445715013
239,2,48516,4.0,1445715064
240,2,58559,4.5,1445715141
241,2,60756,5.0,1445714980


In [3]:
movies = pd.read_csv('./ml-latest/movies.csv', encoding='latin-1')
print (movies.shape)
movies.head()

(58098, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
num_users = ratings.userId.unique().shape[0]
num_items = ratings.movieId.unique().shape[0]
num_users, num_items

(610, 9724)

In [6]:
from sklearn import model_selection as ms
train_set, test_set = ms.train_test_split(ratings, test_size=0.25)
len(train_set), len(test_set)

(75627, 25209)

In [36]:
UM_matrix_ds = ratings.pivot(index='userId', columns='movieId', values='rating')
print("UM Matrix value size", UM_matrix_ds.values.size)
print("ratings value size", ratings.values.size)
print("Sparisity: {:4.2f}%".format(ratings.rating.values.size/UM_matrix_ds.values.size*100))
UM_matrix_ds.head(10)

UM Matrix value size 5931640
ratings value size 403344
Sparisity: 1.70%


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,4.0,5.0,3.0,5.0,4.0,4.0,3.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# similarity
import math
from operator import itemgetter
from scipy.spatial import distance

def distance_cosine(a,b):
    return 1-distance.cosine(a,b)

def disance_corr(a,b):
    return 1-distance.correlation(a,b)

def distance_euclidean(a,b):
    return 1/(distance.euclidean(a,b)+1)

In [9]:
def nearest_neighbor_user(user, topN, simFunc) :  
    u1 = UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = {}
    
    # Brute-Force Compute
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        if uid==user:
            continue
            
        for i in ratedIndex:
            if False==math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        # at least 3 intersection items
        if interSectionLen < 3 :
            continue
            
        # similarity functon
        sim = simFunc(interSectionU1, interSectionU2)
        
        if  math.isnan(sim) == False:
            nn[uid] = sim
            
    # top N returned         
    return sorted(nn.items(),key=itemgetter(1),reverse=True)[:(topN+1)]
    #return sorted(nn.items(),key=itemgetter(1))[:-(topN+1):-1]

In [44]:
def nearest_neighbor_item(movie, topN, simFunc):
    m1 = UM_matrix_ds[movie].dropna()
    user_index = m1.index
    nn = {}
    
    for movieId, col in UM_matrix_ds.iteritems():
        interSectionU1 = []
        interSectionU2 = []
        if movie == movieId:
            continue
        
        for i in user_index:
            if False==math.isnan(col[i]):
                interSectionU1.append(m1[i])
                interSectionU2.append(col[i])
        interSectionLen = len(interSectionU1)
        # at least 3 intersection items
        if interSectionLen < 3 :
            continue
            
        # similarity functon
        sim = simFunc(interSectionU1, interSectionU2)
        
        if  math.isnan(sim) == False:
            nn[movieId] = sim
            
    # top N returned         
    return sorted(nn.items(),key=itemgetter(1),reverse=True)[:(topN+1)]
        
        

In [53]:
print(pd.DataFrame(nearest_neighbor_user(278, 100, distance_euclidean), columns=['userId', 'sim']))
print(pd.DataFrame(nearest_neighbor_item(1, 100, distance_euclidean), columns=['movieId', 'sim']))

     userId       sim
0         8  1.000000
1        15  1.000000
2       348  1.000000
3       352  1.000000
4       373  1.000000
5       382  1.000000
6       512  1.000000
7       566  1.000000
8       601  1.000000
9       602  1.000000
10        1  0.666667
11      296  0.666667
12      338  0.666667
13      484  0.666667
14      515  0.666667
15      572  0.666667
16       17  0.585786
17       59  0.585786
18       72  0.585786
19      178  0.585786
20      232  0.585786
21      254  0.585786
22      280  0.585786
23      282  0.585786
24      330  0.585786
25      343  0.585786
26      415  0.585786
27      418  0.585786
28      489  0.585786
29      587  0.585786
..      ...       ...
71      200  0.387426
72      202  0.387426
73      219  0.387426
74      239  0.387426
75      249  0.387426
76      152  0.376179
77      420  0.376179
78      590  0.376179
79      356  0.366025
80      105  0.356789
81      318  0.356789
82      434  0.356789
83      452  0.356789
84      55

In [65]:
def predictRating(id_num, nn=50, simFunc=distance_euclidean, kind = 'user') :
   
    if (kind == "user"):
        ## neighboorhood 
        neighbor = nearest_neighbor_user(id_num, nn, simFunc)
        # userid: dictionary of similarity

        neighbor_id = [id for id, sim in neighbor]
        # List of similar similarities

        ## Remove if four or more are NaN
        neighbor_movie = UM_matrix_ds.loc[neighbor_id].dropna(1, how = 'all', thresh = 4)
        # User id x DataFrame of movie id. However, if the NaN of the column is 4 or more, it is deleted.

        neighbor_dic = (dict(neighbor))
        ret = [] #['movieId', 'predictedRate']

        # iterate through each column key: userid, column: movieid, value: rating
        for movieId, row in neighbor_movie.iteritems():
            jsum, wsum = 0, 0
            for v in row.dropna().iteritems():
                sim = neighbor_dic.get(v[0],0)
                jsum += sim
                wsum += (v[1]*sim)
            ret.append([movieId, wsum/jsum])

        return ret
    
    elif (kind == 'item'):
        neighbor = nearest_neighbor_item(id_num, nn, simFunc)
        
        neighbor_id = [id for id, sim in neighbor]
        
        neighbor_ratedUser = UM_matrix_ds[neighbor_id].dropna(1, how = 'all', thresh = 4)
        
        neighbor_dic = dict(neighbor)
        ret = []
        
        for userId, row in neighbor_ratedUser.iterrows():
            jsum, wsum = 0, 0
            for v in row.dropna().iteritems():
                sim = neighbor_dic.get(v[0], 0)
                jsum += sim
                wsum += (v[1]*sim)
            ret.append([userId, wsum/jsum])
        
        return ret
        

In [66]:
# predictRating(10, 50, kind = 'user')
predictRating(1, 50, kind = 'item')

movieId  2940    2967    3111    3384    4745    5876    7766    482     \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
6           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
7           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
8           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
11          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
12          NaN     NaN  

ZeroDivisionError: division by zero

In [54]:
## view rated movies and movies with high ratings
def ratingMovies(user_id):
    ds = pd.merge(ratings[ratings.userId==user_id], movies, on=['movieId'])
    return ds.sort_values(by=['rating'], ascending=False)[['rating','title','genres','movieId']].head(100)
ratingMovies(1).head()

,rating,title,genres,movieId
231,5.0,M*A*S*H (a.k.a. MASH) (1970),Comedy|Drama|War,5060
185,5.0,Excalibur (1981),Adventure|Fantasy,2872
89,5.0,Indiana Jones and the Last Crusade (1989),Action|Adventure,1291
90,5.0,Pink Floyd: The Wall (1982),Drama|Musical,1298
190,5.0,From Russia with Love (1963),Action|Adventure|Thriller,2948


In [55]:
def join_movie_info(predicted_result):
    predicted_ratings = pd.DataFrame(predicted_result, columns=['movieId', 'predicted_rating'])
    result_ds = pd.merge(movies[movies.movieId > 0], predicted_ratings, on=['movieId'])
    return result_ds.sort_values(by=['predicted_rating'], ascending=False)

result_user = predictRating(2);
join_movie_info(result).head(10)

,movieId,title,genres,predicted_rating
433,2360,"Celebration, The (Festen) (1998)",Drama,4.899180
561,3328,Ghost Dog: The Way of the Samurai (1999),Crime|Drama,4.662169
781,7099,NausicaÃ¤ of the Valley of the Wind (Kaze no t...,Adventure|Animation|Drama|Fantasy|Sci-Fi,4.629608
239,1244,Manhattan (1979),Comedy|Drama|Romance,4.610808
699,5225,And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001),Drama|Romance,4.609613
375,2010,Metropolis (1927),Drama|Sci-Fi,4.602877
222,1212,"Third Man, The (1949)",Film-Noir|Mystery|Thriller,4.587203
65,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.585418
427,2324,Life Is Beautiful (La Vita Ã¨ bella) (1997),Comedy|Drama|Romance|War,4.582882
194,1090,Platoon (1986),Drama|War,4.545359


In [15]:
# 5th user's star rating
userId=1
pd.merge(ratingMovies(userId), join_movie_info(predictRating(userId)), 
         on=['movieId'], how='right').sort_values(by=['predicted_rating'], ascending =False)

,rating,title_x,genres_x,movieId,title_y,genres_y,predicted_rating
3,5.0,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,5.000000
12,NaN,NaN,NaN,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,5.000000
0,5.0,American Beauty (1999),Drama|Romance,2858,American Beauty (1999),Drama|Romance,5.000000
13,NaN,NaN,NaN,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,4.886699
6,5.0,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,4.866430
14,NaN,NaN,NaN,3147,"Green Mile, The (1999)",Crime|Drama,4.846440
8,5.0,Gladiator (2000),Action|Adventure|Drama,3578,Gladiator (2000),Action|Adventure|Drama,4.809555
15,NaN,NaN,NaN,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,Adventure|Animation|Fantasy,4.775069
11,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,4.771741
16,NaN,NaN,NaN,112552,Whiplash (2014),Drama,4.764554


In [16]:
eval_ratings = ratings
# ratings['userId'].drop_duplicates().values[:]
def eval_prediction(predict_users, n_users=50):
    ## evaluation
    ds = pd.merge(eval_ratings, ratings[['movieId','rating']].groupby(['movieId']).mean().reset_index(), 
                  on='movieId', how='left')

    ds = ds.rename(columns= {'rating_x':'rating', 'rating_y':'mean_rating'})

    st = time.time()
    ## udpate to predict_rating 
    distance_functions = [('euclidean',distance_euclidean), ('cosine', distance_cosine)]
    for name, func in distance_functions:
        ds[name] = 0
        for user_id in predict_users:
            # key: movie id, value : predicted_rating
            for x in predictRating(user_id, n_users, func):
                ds.loc[(ds.userId==user_id) & (ds.movieId==x[0]),name]=x[1]
    print('runtime', round(time.time()-st,2), 'sec')
    #Only the tuple corresponding to the user performing the predict in the entire dataFrame is returned
    return ds[ds.euclidean+ds.cosine>0]

In [17]:
## all userId list 
users = UM_matrix_ds.index.tolist()
## 10-person star rating
predicted = eval_prediction(users[:50], 50)
predicted

runtime 123.91 sec


,userId,movieId,rating,timestamp,mean_rating,euclidean,cosine
0,1,1,4.0,964982703,3.920930,4.083103,3.750041
3,1,47,5.0,964983815,3.975369,4.771741,4.500235
4,1,50,5.0,964982931,4.237745,4.685336,4.444768
7,1,110,4.0,964982176,4.031646,4.300063,4.332735
15,1,260,5.0,964981680,4.231076,4.760956,4.433199
16,1,296,3.0,964982967,4.197068,3.496495,3.999294
20,1,356,4.0,964980962,4.164134,4.386421,4.249997
25,1,457,5.0,964981909,3.992105,0.000000,4.600029
28,1,527,5.0,964984002,4.225000,4.763479,4.684109
32,1,590,4.0,964982546,3.835366,0.000000,4.286133


In [18]:
predicted = predicted[ (predicted['cosine'] > 0) & (predicted['euclidean'] > 0) ]
def RMSE(X, left_col, right_col):
    return(np.sqrt(np.mean((X[left_col] - X[right_col])**2)))

def MAE(X, left_col, right_col):
    return(np.mean(np.absolute(X[left_col] - X[right_col])) )

for name in ['mean_rating', 'cosine', 'euclidean']:
    print ("MAE of {0} is {1} ".format(name, MAE(predicted, 'rating', name)))

for name in ['mean_rating', 'cosine', 'euclidean']:
    print ("RMSE of {0} is {1} ".format(name, RMSE(predicted, 'rating', name)))

MAE of mean_rating is 0.6732807276979298 
MAE of cosine is 0.5531792589753918 
MAE of euclidean is 0.40421532736958293 
RMSE of mean_rating is 0.8576843201841777 
RMSE of cosine is 0.7377415273102896 
RMSE of euclidean is 0.5665102964641624 
